In [3]:
import json
import os
import pandas as pd

In [4]:
data_dir="C:/Users/TEMP/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Claire_LWL&EF"

In [5]:
os.chdir(data_dir)

In [6]:
tdf= pd.read_csv('Baby Toolbox_Data_Export.tsv', sep='\t') 

C:\Users\TEMP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (28,34,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
tdf.shape

(107784, 380)

In [8]:
keep = [col for col in tdf.columns if 'AOI' in col]
keep = [col for col in keep if 'Calib' not in col]
print(keep)

['AOI[WholeScreen]Hit', 'AOI[WholeScreen]Hit.1', 'AOI[WholeScreen]Hit.2', 'AOI[WholeScreen]Hit.3', 'AOI[LeftStim_Shoulder]Hit', 'AOI[RightStim_Hose]Hit', 'AOI[WholeScreen]Hit.4', 'AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5', 'AOI[LeftStim_Rooster]Hit', 'AOI[RightStim_Playground]Hit', 'AOI[WholeScreen]Hit.6', 'AOI[LeftStim_Bike]Hit', 'AOI[RightStim_Sneaker]Hit', 'AOI[WholeScreen]Hit.7', 'AOI[LeftStim]Hit', 'AOI[RightStim]Hit', 'AOI[WholeScreen]Hit.8', 'AOI[LeftStim]Hit.1', 'AOI[RightStim]Hit.1', 'AOI[WholeScreen]Hit.9', 'AOI[LeftStim]Hit.2', 'AOI[RightStim]Hit.2', 'AOI[WholeScreen]Hit.10', 'AOI[LeftStim]Hit.3', 'AOI[RightStim]Hit.3', 'AOI[WholeScreen]Hit.11', 'AOI[LeftStim]Hit.4', 'AOI[RightStim]Hit.4', 'AOI[WholeScreen]Hit.12', 'AOI[LeftStim]Hit.5', 'AOI[RightStim]Hit.5', 'AOI[WholeScreen]Hit.13', 'AOI[LeftStim]Hit.6', 'AOI[RightStim]Hit.6', 'AOI[WholeScreen]Hit.14', 'AOI[LeftStim]Hit.7', 'AOI[RightStim]Hit.7', 'AOI[WholeScreen]Hit.15', 'AOI[LeftStim]Hi

In [9]:
k24 = keep[4:76]

In [10]:
tdf=tdf[k24]

In [11]:
tdata = round(tdf.mean(),4).to_dict()

In [12]:
test = tdf[tdf['AOI[WholeScreen]Hit.4'].notnull()][['AOI[LeftStim_Shoulder]Hit',
 'AOI[RightStim_Hose]Hit',
 'AOI[WholeScreen]Hit.4']]

In [13]:
left =[]
right=[]
away =[]
onscreen = []
i = 0
for val in tdata:
    if i%3 == 0:
        rv = tdata[val]
        left.append(tdata[val])
    elif i%3 == 1:
        rv += tdata[val]
        right.append(tdata[val])
    else:
        rv = 1 - rv
        away.append(rv)
        onscreen.append(tdata[val])
    i+=1

In [14]:
k24[3:6]

['AOI[LeftStim_Ladder]Hit', 'AOI[RightStim_Melon]Hit', 'AOI[WholeScreen]Hit.5']

In [15]:
test

,AOI[LeftStim_Shoulder]Hit,AOI[RightStim_Hose]Hit,AOI[WholeScreen]Hit.4
3847,0.0,0.0,1.0
3848,0.0,0.0,1.0
3849,0.0,0.0,1.0
3850,0.0,0.0,1.0
3851,0.0,0.0,1.0
...,...,...,...
4884,1.0,0.0,1.0
4885,1.0,0.0,1.0
4886,1.0,0.0,1.0
4887,1.0,0.0,1.0


In [16]:
test['AOI[LeftStim_Shoulder]Hit'].sum()

541.0

In [17]:
test[(test['AOI[LeftStim_Shoulder]Hit'] == 0) & (test['AOI[RightStim_Hose]Hit']==0)&(test['AOI[WholeScreen]Hit.4']==1)]

,AOI[LeftStim_Shoulder]Hit,AOI[RightStim_Hose]Hit,AOI[WholeScreen]Hit.4
3847,0.0,0.0,1.0
3848,0.0,0.0,1.0
3849,0.0,0.0,1.0
3850,0.0,0.0,1.0
3851,0.0,0.0,1.0
...,...,...,...
4298,0.0,0.0,1.0
4299,0.0,0.0,1.0
4300,0.0,0.0,1.0
4301,0.0,0.0,1.0


# json file

In [18]:
data_dir="C:/Users/TEMP/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire/"

In [19]:
os.chdir(data_dir)

In [20]:
newfile ='949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00025@2021-05-11-22.21.08.674@ExecutiveFunction@Claire@VerboseGazeData.json'

In [21]:
#df=pd.read_json(newfile)

In [22]:
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [23]:
cols=['itemID','eventName','gazeLocationName', 'gazeLocationOnScreen']

In [24]:
df=df[cols]

In [25]:
df.eventName.unique()

array(['beganCalibration', 'data', 'beganCalibrationRound1',
       'faceVerticesChanged', 'presentedCalibrationItem',
       'cameraImageCaptured', 'lookDetected', 'completedCalibrationItem',
       'calibrationAlignmentCheckPassed', 'completedCalibration',
       'beganLiveTest', 'presentedLiveItem', 'gazeLocationChanged',
       'audioStarted', 'audioCompleted', 'beganLookDetection',
       'endedLookDetection', 'completedLiveItem', 'presentedFillerScreen',
       'removedFillerScreen', 'faceLeftScene', 'faceReturnedToScene',
       'completedLiveTest', 'beganDriftPresentationRound',
       'presentedDriftCalibrationItem', 'completedDriftCalibrationItem',
       'completedDriftPresentationRound', 'instrumentCompleted'],
      dtype=object)

In [26]:
len(df.gazeLocationOnScreen.value_counts())

2

In [27]:
#Finding the index for presentLiveItem and completeLiveItem to subset the pieces we need for analysis
begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)

In [28]:
end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

In [29]:
expected = pd.DataFrame(
    {'Left': left,
     'Right': right,
     'Away': away,'Onscreen':onscreen
    })
expected

,Left,Right,Away,Onscreen
0,0.5192,0.3273,0.1535,0.9376
1,0.1833,0.7927,0.0240,0.9760
2,0.5172,0.3267,0.1561,0.8647
3,0.7618,0.1239,0.1143,0.9318
4,0.3719,0.4531,0.1750,0.8667
5,0.4524,0.1931,0.3545,0.7906
6,0.2635,0.5712,0.1653,0.9221
7,0.6243,0.2393,0.1364,0.8783
8,0.3677,0.5094,0.1229,0.9073
9,0.5598,0.2060,0.2342,0.7773


In [30]:
len(end)

24

In [31]:
test = df.iloc[605:1236]

In [32]:
test[test['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()

true    83
Name: gazeLocationOnScreen, dtype: int64

# Fisher Exact Test 

In [60]:
#tsv file
exp = pd.DataFrame(columns=('left', 'right','onScreenAway','offScreenAway'))
for i in range(24):
    lcol,rcol,wcol = k24[3*i:3*i+3]
    
    subset = tdf[tdf[wcol].notnull()]
    
    a=subset[lcol].sum()
    b=subset[rcol].sum()
    d=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==1)].shape[0]
    e=subset[(subset[lcol]==0) & (subset[rcol]==0) &(subset[wcol]==0)].shape[0]

    exp.loc[i] = [a, b ,d, e]

In [79]:
exp.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/tsv_count.csv', index=False)

In [80]:
obs.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/json_count.csv', index=False)

In [35]:
#json file
obs = pd.DataFrame(columns=('L', 'R','onScreenAway','offScreenAway'))
items =[]
for i in range(len(end)):
    sub = df.iloc[begin[i]+1:end[i]]
    items.extend(sub.itemID.dropna().unique())
    a, b, c = sub.gazeLocationName.value_counts()[['left','right','away']].round(4) 
    if len(sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()) == 2:
        d,e = sub[sub['gazeLocationName'] == 'away'].gazeLocationOnScreen.value_counts()
    else:
        d = c
        e = 0
    obs.loc[i] = [a, b ,d, e]

In [72]:
l1 =obs.to_numpy()

In [76]:
items

['Shoulder_Hose_G5',
 'Ladder_Melon_G5',
 'Rooster_Playground_G5',
 'Tricycle_Sneaker_G5',
 'Hose_Shoulder_G5',
 'Melon_Ladder_G5',
 'Playground_Rooster_G5',
 'Sneaker_Tricycle_G5',
 'Star_Moose_G6',
 'Turtle_Hammer_G6',
 'Cake_Bee_G6',
 'Lamp_Stove_G6',
 'Moose_Star_G6',
 'Hammer_Turtle_G6',
 'Bee_Cake_G6',
 'Stove_Lamp_G6',
 'Garden_Sweater_G7',
 'Ankle_Scissors_G7',
 'Woods_Tray_G7',
 'Bench_Rip_G7',
 'Sweater_Garden_G7',
 'Scissors_Ankle_G7',
 'Tray_Woods_G7',
 'Rip_Bench_G7']

In [77]:
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import chi2

for i in range(24):
    data = [l1[i], l2[2]]
    
    stat, p, dof, expected = chi2_contingency(data)
    print(items[i])
    
    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    print('')

Shoulder_Hose_G5
probability=0.950, critical=7.815, stat=205.763
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Ladder_Melon_G5
probability=0.950, critical=7.815, stat=268.473
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Rooster_Playground_G5
probability=0.950, critical=7.815, stat=55.133
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Tricycle_Sneaker_G5
probability=0.950, critical=7.815, stat=148.672
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Hose_Shoulder_G5
probability=0.950, critical=7.815, stat=136.355
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Melon_Ladder_G5
probability=0.950, critical=7.815, stat=183.149
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Playground_Rooster_G5
probability=0.950, critical=7.815, stat=173.240
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)

Sneaker_Tricycle_G5
prob

In [66]:
l2

array([[541., 341.,  95.,  65.],
       [176., 761.,   0.,  23.],
       [497., 314.,  20., 130.],
       [793., 129.,  48.,  71.],
       [357., 435.,  40., 128.],
       [471., 201., 151., 218.],
       [274., 594.,  91.,  81.],
       [600., 230.,  14., 117.],
       [353., 489.,  29.,  89.],
       [538., 198.,  11., 214.],
       [176., 824.,   0.,  41.],
       [381., 440.,  40., 180.],
       [681., 318.,  14.,  28.],
       [308., 483., 139., 110.],
       [706., 244.,  62.,  29.],
       [837.,  63.,   7.,  53.],
       [645., 289.,   0.,  28.],
       [673., 329.,  17.,  21.],
       [768., 194.,  26.,  52.],
       [461., 364.,   6., 130.],
       [543., 475.,   0.,  22.],
       [285., 619.,  56.,  80.],
       [336., 330.,  29., 265.],
       [725.,  70.,  27., 140.]])

In [41]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2
# contingency table
table = [ob,ex]

stat, p, dof, expected = chi2_contingency(table)
print('dof=%d' % dof)

# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')
    
# interpret p-value
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

dof=15
probability=0.950, critical=24.996, stat=0.138
Independent (fail to reject H0)
significance=0.050, p=1.000
Independent (fail to reject H0)


In [142]:
from scipy.stats import chisquare
chisq = pd.DataFrame(columns=('items', 'stat','p'))
for i in range(24):
    ob = obs.iloc[i]
    ex = expected.iloc[i]
    stat, pv = chisquare(f_obs=ob, f_exp=ex)
    chisq.loc[i] = [items[i], round(stat,3) ,round(pv,3)]


In [144]:
chisq.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/chisquare.csv', index=False)

In [145]:
!DIR

 Volume in drive C is Windows
 Volume Serial Number is 1E0A-261A

 Directory of C:\Users\TEMP\Box\Baby Toolbox Operations\Domain Teams\Language\Tobii Pilot\Tobii Verbose Files\Claire

05/24/2021  01:01 PM    <DIR>          .
05/24/2021  01:01 PM    <DIR>          ..
05/11/2021  05:11 PM      (70,162,558) 949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData 2.json
05/11/2021  05:11 PM        70,162,558 949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json
05/11/2021  05:11 PM         4,921,354 949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.53.662@LookListening@Claire@VerboseGazeData 2.csv
05/11/2021  05:11 PM       (4,921,354) 949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.53.662@LookListening@Claire@VerboseGaze

In [147]:
obs.to_csv('C:/Users/TEMP/Box/Baby Toolbox Operations/Analysis/summary.csv', index=False)